In [ ]:
# Version History
#print('Version 1.0.0: 08/26/2022 3:48pm - Nate Calvanese - Moving transformation functions to a shared utility notebook')
#print('Version 1.0.1: 08/30/2022 11:00am - Nate Calvanese - Moving in additional utility functions')
#print('Version 1.0.2: 08/30/2022 11:00am - Nate Calvanese - Moving in FAPI workspace functions')
#print('Version 1.0.3: 09/16/2022 10:46am - Nate Calvanese - Update to find_file_in_inventory function to deal with NaN')
#print('Version 1.0.4: 09/19/2022 5:06pm - Nate Calvanese - Added find_fileref_fields function')
#print('Version 1.0.5: 09/23/2022 4:29pm - Nate Calvanese - Moved in ELT pipeline functions')
#print('Version 1.0.6: 09/26/2022 3:22pm - Nate Calvanese - Added PHS and dataset formatting functions')
#print('Version 1.0.7: 09/26/2022 4:36pm - Nate Calvanese - Added schema reconciliation functionality to TDR dataset patching')
#print('Version 1.0.8: 10/5/2022 2:58pm - Nate Calvanese - Added support for chunking ingest requests')
#print('Version 1.0.9: 10/6/2022 10:35am - Nate Calvanese - Updated TDR utility functions')
#print('Version 1.0.10: 10/12/2022 10:55am - Nate Calvanese - Added support for snapshotting in EL pipeline')
#print('Version 1.0.11: 10/18/2022 9:33am - Nate Calvanese - Fixed a few bugs in various functions')
#print('Version 1.0.12: 10/21/2022 9:56am - Nate Calvanese - Added dangling foreign key resolution and additional logging')
#print('Version 1.0.13: 10/24/2022 1:29pm - Nate Calvanese - Added function to derive project entity name')
#print('Version 1.0.14: 10/26/2022 1:00pm - Nate Calvanese - Updated encode name function to only return lowercase')
#print('Version 1.0.15: 11/4/2022 9:35am - Nate Calvanese - Added file relationship inference to T pipeline')
#print('Version 1.0.16: 11/14/2022 2:37pm - Nate Calvanese - Fixed bug in snapshot readers')
#print('Version 1.0.17: 1/31/2023 11:01am - Nate Calvanese - Updated logging')
#print('Version 1.0.19: 2/23/2023 11:02am - Nate Calvanese - Made addition of auth domains as snapshot readers configurable')
#print('Version 1.0.20: 2/25/2023 2:16pm - Nate Calvanese - Replaced FAPI with Requests to make porting to dev easier')
#print('Version 1.0.21: 2/28/2023 11:38am - Nate Calvanese - Turned on bulkMode for ingest')
#print('Version 1.0.22: 3/3/2023 1:01pm - Nate Calvanese - Tweaks to the TDR job monitoring function')
#print('Version 1.0.23: 3/6/2023 2:24pm - Nate Calvanese - Updates to support remote file references')
#print('Version 1.0.24: 3/8/2023 8:46am - Nate Calvanese - Performance updates to file ref lookup')
print('Version 1.0.25: 3/10/2023 8:46am - Nate Calvanese - Turned on the predictable file IDs dataset creation parameter')



In [ ]:
#!pip install --upgrade import_ipynb data_repo_client

# Imports

In [ ]:
import import_ipynb
import uuid
import pandas as pd
from time import sleep
import requests
import data_repo_client
import json
import re
import logging
import datetime
import sys
import urllib3
import xmltodict
from time import sleep
from google.cloud import storage
from google.cloud import bigquery
import google.auth
import google.auth.transport.requests
import source_files_creation as sfc
import build_file_inventory as bfi
import process_table_data as ptd
import build_mapping_query as bmq
import output_data_validation as odv
import resolve_dangling_foreign_keys as rdfk
import infer_file_relationships as ifr
import identify_supplementary_files as isf

# Configure logging format
while logging.root.handlers:
    logging.root.removeHandler(logging.root.handlers[-1])
logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", datefmt="%m/%d/%Y %I:%M:%S %p", level=logging.INFO, handlers=[logging.StreamHandler(sys.stdout)])


# Transformation Functions

In [ ]:
# Function to convert values to list
def convert_to_list(input_str):
    try:
        output = json.loads(input_str)
        if not isinstance(output, list):
            raise Exception
    except:
        output = [str(input_str)]
    return output

# Function to search for file in file inventory and return the associated fileref object
def find_file_in_inventory(search_input, file_lookup, return_field="file_id", match_multi=True, match_regex="", match_type="partial"):
    # Intialize variables
    fileref_obj = []
    if match_regex == "" or match_regex == None:
        match_regex = ""
    
    # If search_input is a list, attempt to match files for every entry in the list
    if isinstance(search_input, list):
        for search_string in search_input:
            # If exact match, attempt to get from dict
            if match_type == "exact":
                try:
                    if return_field == "file_id":
                        file_id = file_lookup.get(search_string)["file_id"]
                        fileref_obj.append(file_id)
                    else:
                        file_ref = file_lookup.get(search_string)["file_ref"]
                        fileref_obj.append(file_ref)
                except:
                    pass
           
            # Else, loop through file lookup, record fileref_obj entry where matches are found, and return matches
            else:
                if not (search_string == None or pd.isna(search_string)):
                    for entry in file_lookup.keys():
                        if str(search_string) in entry and re.search(match_regex, entry):
                            if return_field == "file_id":
                                fileref_obj.append(file_lookup[entry]["file_id"])
                            else:
                                fileref_obj.append(file_lookup[entry]["file_ref"])
                            if match_multi == False:
                                break
        return fileref_obj
    else:
        search_string = search_input
        # If exact match, attempt to get from dict
        if match_type == "exact":
            try:
                if return_field == "file_id":
                    file_id = file_lookup.get(search_string)["file_id"]
                    fileref_obj.append(file_id)
                else:
                    file_ref = file_lookup.get(search_string)["file_ref"]
                    fileref_obj.append(file_ref)
            except:
                pass
        
        # Else, loop through file inventory, record fileref_obj entry where matches are found, and return matches
        else:
            if not (search_string == None or pd.isna(search_string)):
                for entry in file_lookup.keys():
                    if str(search_string) in entry and re.search(match_regex, entry):
                        if return_field == "file_id":
                            fileref_obj.append(file_lookup[entry]["file_id"])
                        else:
                            fileref_obj.append(file_lookup[entry]["file_ref"])
                        if match_multi == False:
                            break       
        if match_multi == True:
            return fileref_obj
        else:
            if not fileref_obj:
                return None
            else:
                return fileref_obj[0]

# Function to encode table/field names (removing unwanted characters, setting to lower case, etc.)
def encode_name(string):
    reserved_words= ["all", "and","any","array","as","asc","assert_rows_modified","at","between","by","case","cast","collate","contains","create","cross","cube","current","default","define","desc","distinct","else","end","enum","escape","except","exclude","exists","extract","false","fetch","following","for","from","full","group","grouping","groups","hash","having","if","ignore","in","inner","intersect","interval","into","is","join","lateral","left","like","limit","lookup","merge","natural","new","no","not","null","nulls","of","on","or","order","outer","over","partition","preceding","proto","qualify","range","recursive","respect","right","rollup","rows","select","set","some","struct","tablesample","then","to","treat","true","unbounded","union","unnest","using","when","where","window","with","within"]
    out_str = string.strip().lower()
    out_str = re.sub('^entity:', '', out_str) # Specific to the workspace case, should prob be configurable
    out_str = re.sub('[\-\.\-:]', '_', out_str)
    out_str = re.sub('[^a-zA-Z0-9_]', '', out_str)
    if re.match('^[_0-9]+', out_str):
        out_str = 't_' + out_str
    if out_str in reserved_words:
        out_str = 't_' + out_str
    return out_str

# Function to map data types to TDR types
def map_datatype(string):
    lookup = {
        'string': 'string',
        'int': 'integer',
        'int8': 'integer',
        'int16': 'integer', 
        'int32': 'integer', 
        'int64': 'integer', 
        'boolean': 'boolean',
        'float': 'float',
        'float32': 'float',
        'float64': 'float'
    }
    return lookup.get(string.lower(), 'other')

# Function to construct relationship object between two fields
def construct_relationship(src_table, src_column, tar_table, tar_column):
    rel_name = src_table + '_' + src_column + '__to__' + tar_table + '_' + tar_column
    rel_dict = {'name': rel_name, 'from': {'table': src_table, 'column': src_column}, 'to': {'table': tar_table, 'column': tar_column}}
    return rel_dict

# Function to properly format a PHS id
def format_phs_id(input_str):
    try:
        num = re.search("phs0*([0-9]+)", input_str, re.IGNORECASE).group(1)
    except:
        num = ""
    if num:
        output_str = "phs" + str(num).zfill(6)
    else:
        output_str = ""
    return output_str

# Function to build default TDR dataset name (Anvil Specific)
def format_dataset_name(input_str):
    current_datetime = datetime.datetime.now()
    current_date_string = current_datetime.strftime("%Y%m%d")
    output_str = "ANVIL_" + re.sub("^ANVIL[_]?", "", input_str, flags=re.IGNORECASE) + "_" + current_date_string
    output_str = re.sub("[^a-zA-Z0-9_]", "_", output_str)
    return output_str


# Firecloud Utility Functions

In [ ]:
# Function to pull and format workspace attributes
def get_workspace_attributes(ws_project, ws_name):
    # Establish credentials
    creds, project = google.auth.default()
    auth_req = google.auth.transport.requests.Request()
    creds.refresh(auth_req)
    
    # Pull workspace attributes
    ws_attributes = requests.get(
        url=f"https://api.firecloud.org/api/workspaces/{ws_project}/{ws_name}?fields=workspace.attributes,workspace.authorizationDomain,workspace.googleProject,workspace.bucketName",
        headers={"Authorization": f"Bearer {creds.token}"}
    ).json()
    
    # Format workspace attributes (replace nested dictionaries with the lists they represent, for example)
    for key, val in ws_attributes["workspace"]["attributes"].items():
        if isinstance(val, dict):
            if val.get("items") != None:
                ws_attributes["workspace"]["attributes"][key] = val["items"]
    
    # Add additional attributes for PHS ID and Workspace Name
    ws_attributes["workspace"]["project"] = ws_project
    ws_attributes["workspace"]["name"] = ws_name
    ws_attributes["workspace"]["attributes"]["phs_id"] = ""
    if ws_attributes["workspace"]["attributes"].get("tag:tags"):
        for item in ws_attributes["workspace"]["attributes"]["tag:tags"]:
            if 'dbgap' in item.lower(): 
                ws_attributes["workspace"]["attributes"]["phs_id"] = item.lower().split(":", 1)[1].strip()
    return ws_attributes["workspace"]

# Function to get list of entity types in a workspace
def list_entity_types(ws_project, ws_name):
    # Establish credentials
    creds, project = google.auth.default()
    auth_req = google.auth.transport.requests.Request()
    creds.refresh(auth_req)
    
    # Get and return list of entity types
    response = requests.get(
        url=f"https://api.firecloud.org/api/workspaces/{ws_project}/{ws_name}/entities",
        headers={"Authorization": f"Bearer {creds.token}"}
    )
    return response

# Function to return entities of an entity type in a workspace
def get_entities(ws_project, ws_name, etype):
    # Establish credentials
    creds, project = google.auth.default()
    auth_req = google.auth.transport.requests.Request()
    creds.refresh(auth_req)

    # Get and return entities of specified type
    response = requests.get(
            url=f"https://api.firecloud.org/api/workspaces/{ws_project}/{ws_name}/entities/{etype}",
            headers={"Authorization": f"Bearer {creds.token}"}
        )
    return response

# Function to collect remote file references from a file ref field
def collect_remote_filerefs(ws_project, ws_name, entity, field, data_files_src_bucket):
    response_entities = get_entities(ws_project, ws_name, entity)
    entities_dict = json.loads(response_entities.text)
    # Loop through entities and add items to the data_files_src_bucket dict as necessary
    for row in entities_dict:
        try:
            for key, val in row["attributes"].items():
                if key == field:
                    if not isinstance(val, dict):
                        bucket = re.match('gs:\/\/([a-z0-9\-]+)\/', val).group(1)
                        obj = re.match('gs:\/\/[a-z0-9\-]+\/([A-Za-z0-9\-_\/\.]+)', val).group(1)
                        if bucket not in (data_files_src_bucket):
                            data_files_src_bucket[bucket] = {"include_dirs": [obj], "exclude_dirs": []}
                        else:
                            if obj not in data_files_src_bucket[bucket]["include_dirs"]:
                                data_files_src_bucket[bucket]["include_dirs"].append(obj)
                    else:
                        if val.get("items"):
                            for entry in val["items"]:
                                bucket = re.match('gs:\/\/([a-z0-9\-]+)\/', entry).group(1)
                                obj = re.match('gs:\/\/[a-z0-9\-]+\/([A-Za-z0-9\-_\/\.]+)', entry).group(1)
                                if bucket not in (data_files_src_bucket):
                                    data_files_src_bucket[bucket] = {"include_dirs": [obj], "exclude_dirs": []}
                                else:
                                    if obj not in data_files_src_bucket[bucket]["include_dirs"]:
                                        data_files_src_bucket[bucket]["include_dirs"].append(obj)
        except:
            break
    return data_files_src_bucket

# Function to scan entity fields for file references and add to the list of file ref fields to consider if found
def find_and_add_fileref_fields(ws_project, ws_name, data_files_bucket, data_file_refs_dict, data_files_src_bucket, rows_to_scan=100):
    # Pre-process data_file_refs_dict
    data_file_refs_set = set()
    for key, val in data_file_refs_dict.items():
        for column_entry in val:
            data_file_refs_set.add(key + "." + column_entry["column"])
    
    # Establish credentials
    creds, project = google.auth.default()
    auth_req = google.auth.transport.requests.Request()
    creds.refresh(auth_req)
    
    # Get list of entity types
    response_etypes = list_entity_types(ws_project, ws_name)
    dict_all_etypes = json.loads(response_etypes.text)
    etypes_list = [key for key in dict_all_etypes.keys()]
    
    # Loop through entity types and check for file references (both local and remote)
    file_path = "gs://" + data_files_bucket
    fileref_attr_set = set()
    remote_bucket_set = set()
    for etype in etypes_list:
        response_entities = get_entities(ws_project, ws_name, etype)
        entities_dict = json.loads(response_entities.text) 
        for i in range(0, rows_to_scan):
            try:
                for key, val in entities_dict[i]["attributes"].items():
                    if not isinstance(val, dict):
                        if file_path in val:
                            fileref_attr_set.add(etype + "." + key)
                        elif re.search("^gs:\/\/([a-z0-9\.\-_]+)\/", val):
                            remote_bucket_set.add(re.search("^gs:\/\/([a-z0-9\.\-_]+)\/", val).group(1))
                            fileref_attr_set.add(etype + "." + key)
                            data_files_src_bucket = collect_remote_filerefs(ws_project, ws_name, etype, key, data_files_src_bucket)
                    else:
                        if val.get("items"):
                            for entry in val["items"]:
                                if file_path in entry:
                                    fileref_attr_set.add(etype + "." + key)
                                elif re.search("^gs:\/\/([a-z0-9\.\-_]+)\/", entry):
                                    remote_bucket_set.add(re.search("^gs:\/\/([a-z0-9\.\-_]+)\/", entry).group(1))
                                    fileref_attr_set.add(etype + "." + key)
                                    data_files_src_bucket = collect_remote_filerefs(ws_project, ws_name, etype, key, data_files_src_bucket)
            except:
                break
    
    # Determine if field is already in data_file_refs dictionary and add if not
    final_fileref_attr_set = set()
    if fileref_attr_set:
        for fileref in fileref_attr_set:
            if fileref not in data_file_refs_set:
                final_fileref_attr_set.add(fileref)
                file = fileref.split(".")[0]
                column = fileref.split(".")[1]
                column_dict = {
                    "column": column,
                    "method": "file_path_match",
                    "match_multiple_files": False, 
                    "match_regex": None,
                    "match_type": "exact",
                    "mode": "fileref_in_line",
                    "create_new_field": False,
                    "new_field_name": None
                }
                if data_file_refs_dict.get(file):
                    data_file_refs_dict[file].append(column_dict)
                else:
                    data_file_refs_dict[file] = [column_dict]
    else:
         final_fileref_attr_set = fileref_attr_set
    return list(final_fileref_attr_set), data_file_refs_dict, data_files_src_bucket, list(remote_bucket_set)   

# Function to add user to Terra group
def add_user_to_group(group, role, email):
    # Establish credentials
    creds, project = google.auth.default()
    auth_req = google.auth.transport.requests.Request()
    creds.refresh(auth_req)
    
    # Add user to group and return response
    response = requests.put(
        url=f"https://api.firecloud.org/api/groups/{group}/{role}/{email}",
        headers={"Authorization": f"Bearer {creds.token}"}
    )
    return response

# Function to update workspace access list
def update_workspace_acl(ws_project, ws_name, acl_updates):
    # Establish credentials
    creds, project = google.auth.default()
    auth_req = google.auth.transport.requests.Request()
    creds.refresh(auth_req)
    
    # Add user to group and return response
    response = requests.patch(
        url=f"https://api.firecloud.org/api/workspaces/{ws_project}/{ws_name}/acl",
        headers={"Authorization": f"Bearer {creds.token}"},
        json=acl_updates
    )
    return response


# TDR Utility Functions

In [ ]:
# Function to refresh TDR API client
def refresh_tdr_api_client():
    creds, project = google.auth.default()
    auth_req = google.auth.transport.requests.Request()
    creds.refresh(auth_req)
    config = data_repo_client.Configuration()
    config.host = "https://data.terra.bio"
    config.access_token = creds.token
    api_client = data_repo_client.ApiClient(configuration=config)
    api_client.client_side_validation = False
    return api_client

# Function to wait for TDR job completion
def wait_for_tdr_job(job_model):
    result = job_model
    print("TDR Job ID: " + job_model.id)
    counter = 0
    conn_err_counter = 0
    while True:
        # Re-establish credentials and API clients every 30 minutes
        if counter == 0 or counter%180 == 0:
            api_client = refresh_tdr_api_client()
            jobs_api = data_repo_client.JobsApi(api_client=api_client)
        # Check for TDR connectivity issues and raise exception if issue persists
        if result == None or result.status_code in ["500", "502", "503"]:
            conn_err_counter += 1
            if conn_err_counter < 5:
                sleep(10)
                counter += 1
                attempt_counter = 0
                while True:
                    try:
                        result = jobs_api.retrieve_job(job_model.id)
                        break
                    except Exception as e:
                        sleep(10)
                        attempt_counter += 1
                        if attempt_counter < 5:
                            continue
                        else:
                            raise "Error retrieving job status: {}".format(str(e)) 
            else:
                raise "Error interacting with TDR: {}".format(result.status_code)
        # Check if job is still running, and sleep/re-check if so
        elif result == None or result.job_status == "running":
            conn_err_counter = 0
            sleep(10)
            counter += 1
            attempt_counter = 0
            while True:
                try:
                    result = jobs_api.retrieve_job(job_model.id)
                    break
                except Exception as e:
                    sleep(10)
                    attempt_counter += 1
                    if attempt_counter < 5:
                        continue
                    else:
                        raise "Error retrieving job status: {}".format(str(e))
        elif result.job_status == "failed":
            try:
                fail_result = jobs_api.retrieve_job_result(job_model.id)
                raise Exception("Job " + job_model.id + " failed: " + fail_result)
            except Exception as e:
                raise Exception("Job " + job_model.id + " failed: " + str(e))
        elif result.job_status == "succeeded":
            attempt_counter = 0
            while True:
                try:
                    return jobs_api.retrieve_job_result(job_model.id), job_model.id
                except Exception as e:
                    sleep(10)
                    attempt_counter += 1
                    if attempt_counter < 3:
                        continue
                    else:
                        return "Job succeeded, but error retrieving job result: {}".format(str(e)), job_model.id
        else:
            raise "Unrecognized job state: {}".format(result.job_status)
        

# "EL" Pipeline Functions

In [1]:
# Function to copy log results to the cloud
def copy_log_results(logs_destination_dir, logs_stream_file_path, logs_table_file_path, df_results):
    # Copy stream file to cloud
    !gsutil cp $logs_stream_file_path $logs_destination_dir/ 2> stdout
    !rm $logs_stream_file_path
    
    # Copy table file to cloud
    df_results.to_csv(logs_table_file_path, index=False, sep="\t")
    !gsutil cp $logs_table_file_path $logs_destination_dir/ 2> stdout
    !rm $logs_table_file_path

# Function to look up dbgap consent codes
def lookup_consent_code(params, phs_id, consent_name):
    logging.info("Attempting to lookup consent code using PHS: {} and Consent Name: {}.".format(phs_id, consent_name))
    # Attempt to read source files into data frame, checking for missing file
    src_file_path = params["ws_bucket"] + "/ingest_pipeline/resources/consent_code_lookup/dbgap_consents.csv"
    try:
        df = pd.read_csv(src_file_path, delimiter = ',')
    except:
        err = "Consent code lookup file not found at: {}".format(src_file_path)
        logging.error(err)
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Snapshot Creation", "Lookup Consent Code", "Warning", err])
        return consent_name
    # Map consent name to consent code and return string
    df2 = df[(df.phs == phs_id) & (df.consent_short_name == consent_name)]
    if len(df2) > 0:
        consent_code = "c" + str(df2["consent_code"].values[0])
        return consent_code
    else:
        return consent_name
            
# Function to create and share a snapshot            
def create_and_share_snapshot(params, src=""):
    logging.info("Creating full-view snapshot.")
    if src == "el_pipeline":
        log_entity = params["staging_area_name"]
    elif src == "t_pipeline":
        log_entity = params["src_dataset"]
    else:
        log_entity = "Unknown"
    
    # Attempt to collect and map consent code
    try:
        phs_short_id = re.search('([0-9]{4,6})', params["phs_id"]).group(1)
    except:
        phs_short_id = ""
    if len(params["consent_name"]) > 0 and len(phs_short_id) > 0:
        consent_code = lookup_consent_code(params, int(phs_short_id), params["consent_name"])
    else:
        logging.warning("Unable to lookup consent code. Consent name and/or PHS ID missing for lookup.")
        consent_code = params["consent_name"]
        params["pipeline_results"].append([log_entity, datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Snapshot Creation", "Lookup Consent Code", "Warning", "Consent name and/or PHS ID missing for lookup"])
    
    # Create reader list for snapshot, including auth domain(s) on WS
    reader_list = []
    snapshot_reader_list = params["snapshot_readers_list"]
    for srl_entry in snapshot_reader_list:
        reader_list.append(srl_entry)
    if "auth-domain" in snapshot_reader_list:
        auth_domain_list = params["auth_domains"] 
        for ad_entry in auth_domain_list:
            reader_list.append(ad_entry + "@firecloud.org")
    
    # Create and submit snapshot creation request
    logging.info("Submitting snapshot request.")
    snapshot_req = {
        "name": params["snapshot_name"],
        "description": "Full view snapshot of " + params["dataset_name"],
        "consentCode": consent_code,
        "contents": [{
            "datasetName": params["dataset_name"],
            "mode": "byFullView"
        }],
        "policies": {
            "stewards": ["anvil_tdr_ingest@firecloud.org"],
            "readers": reader_list 
        },
        "profileId": params["profile_id"]
    }
    attempt_counter = 0
    while True:
        try:
            api_client = refresh_tdr_api_client()
            snapshots_api = data_repo_client.SnapshotsApi(api_client=api_client)
            create_snapshot_result, job_id = wait_for_tdr_job(snapshots_api.create_snapshot(snapshot=snapshot_req))
            logging.info("Snapshot Creation succeeded: {}".format(create_snapshot_result))
            params["pipeline_results"].append([log_entity, datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Snapshot Creation", "Create and Share Snapshot", "Success", "Job_ID: {} - Truncated Response: {}".format(job_id, str(create_snapshot_result)[0:1000])])
            break
        except Exception as e:
            logging.error("Error on Snapshot Creation: {}".format(str(e)))
            attempt_counter += 1
            if attempt_counter < 2:
                logging.info("Retrying Snapshot Creation (attempt #{})...".format(str(attempt_counter)))
                sleep(10)
                continue
            else:
                logging.error("Maximum number of retries exceeded. Recording error to pipeline results.")
                params["pipeline_results"].append([log_entity, datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Snapshot Creation", "Create and Share Snapshot", "Error", str(e)])
                break

# Function to determine whether a specified file exists in cloud storage
def file_exists(key, params):
    logging.info("Checking for file {}...".format(key))
    storage_client = storage.Client()
    bucket = storage_client.bucket(params["ws_bucket_name"])
    return storage.Blob(bucket=bucket, name=key).exists(storage_client)

# Function to find and execute ingests
def run_el_ingests(dataset_id, params, ingest_list):
    
    # Setup/refresh TDR clients
    api_client = refresh_tdr_api_client()
    datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
    
    # If no ingest list specified, create a list of all target tables and files
    if not ingest_list:
        logging.info("Ingest list from table_data_processing step or ingest_list_override parameter is empty. Attempting to build ingest list from target dataset schema.")
        try:
            dataset_info = datasets_api.retrieve_dataset(id=dataset_id, include=["SCHEMA"]).to_dict()
            table_list = [tab["name"] for tab in dataset_info["schema"]["tables"]]
            ingest_list = {}
            for table in table_list:
                ingest_list[table] = []
        except:
            logging.error("Error building ingest from target dataset schema. No ingests will be run.")
            params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Ingests", "Table: All - File: All", "Error", "Ingest list from table_data_processing step or ingest_list_override parameter was empty, and attempts to build a list from the target dataset failed. Skipping ingests."])
            ingest_list = {}
            
    # Update ingest list with files to process (if not specified)
    for key, val in ingest_list.items():
        if val == []:
            directory = params["el_output_dir"] + "/" + key + "/"
            obj_list = bfi.get_objects_list(params["ws_bucket_name"], params["google_project"], dirs_to_include=[directory])
            file_list = []
            for item in obj_list:
                path_split = item.split("/")
                file_name_idx = len(path_split)-1
                file_name = path_split[file_name_idx]
                file_list.append(file_name)
            ingest_list[key] = file_list
    
    # Loop through target tables and run ingest
    for key, val in ingest_list.items():
        target_table = key
        logging.info("Running ingests for target table: {}".format(target_table))
        if val:
            for file_entry in val:
                source_file_name = file_entry
                source_rel_file_path = "{}/{}/{}".format(params["el_output_dir"], target_table, source_file_name)
                source_full_file_path = "{}/{}/{}/{}".format(params["ws_bucket"], params["el_output_dir"], target_table, source_file_name)
                if file_exists(source_rel_file_path, params):
                    logging.info("Running ingest from {} to table {}.".format(source_file_name, target_table))
                    ingest_request = {
                        "table": target_table,
                        "profile_id": params["profile_id"],
                        "ignore_unknown_values": True,
                        "resolve_existing_files": True,
                        "updateStrategy": "replace",
                        "format": "json",
                        "bulkMode": True,
                        "load_tag": "Ingest for {}".format(params["staging_area_name"]),
                        "path": source_full_file_path
                    }
                    attempt_counter = 0
                    while True:
                        try:
                            api_client = refresh_tdr_api_client()
                            datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
                            ingest_request_result, job_id = wait_for_tdr_job(datasets_api.ingest_dataset(id=dataset_id, ingest=ingest_request))
                            logging.info("Ingest from file {} succeeded: {}".format(source_file_name, str(ingest_request_result)[0:1000]))
                            params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Ingests", f"Table: {target_table} - File: {source_file_name}", "Success", "Job_ID: {} - Truncated Response: {}".format(job_id, str(ingest_request_result)[0:1000])])
                            break
                        except Exception as e:
                            logging.error("Error on Dataset Ingest: {}".format(str(e)))
                            attempt_counter += 1
                            if attempt_counter < 2:
                                logging.info("Retrying Dataset Ingest (attempt #{})...".format(str(attempt_counter)))
                                sleep(10)
                                continue
                            else:
                                logging.error("Maximum number of retries exceeded. Recording error to pipeline results.")
                                params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Ingests", f"Table: {target_table} - File: {source_file_name}", "Error", str(e)])
                                break
                else:
                    logging.warning("Source table data file does not exist.  Skipping: {}.".format(source_file_name))
                    params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Ingests", f"Table: {target_table} - File: {source_file_name}", "Warning", f"Source table data file does not exist.  Skipping: {source_file_name}"])
                    continue
        else:
            logging.warning("No source table data files found. Skipping to next target table.")

# Function to add TDR dataset SA to the appropriate Terra groups and workspace ACLs
def set_up_dataset_ingest_sa(dataset_id, params, new_dataset):
    logging.info("Setting up dataset ingest service account (SA).")
    # Setup/refresh TDR clients
    api_client = refresh_tdr_api_client()
    datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
    
    # Collect dataset-specific SA from TDR
    dataset_details = datasets_api.retrieve_dataset(id=dataset_id)
    service_account = dataset_details.ingest_service_account
    if not service_account:
        error_message = "No dataset ingest service account found. Ensure the dedicatedIngestServiceAccount parameter has been set to True on dataset creation."
        logging.error(error_message)
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Service Account Setup", "Add SA to Anvil Ingest Group", "Error", error_message])
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Service Account Setup", "Add SA to Workspace", "Error", error_message])
    else:
        # Add SA user to the Anvil Ingest Terra Group
        anvil_ingest_sa_group = "anvil_tdr_ingest" # full email: anvil_tdr_ingest@firecloud.org
        attempt_counter = 0
        while True:
            try:
                res = add_user_to_group(anvil_ingest_sa_group, "member", service_account)
                if res.status_code == 204:
                    logging.info("Dataset service account {} added to {} sucessfully.".format(service_account, anvil_ingest_sa_group))
                    params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Service Account Setup", "Add SA to Anvil Ingest Group", "Success", service_account])
                    break
                else:
                    if attempt_counter < 2:
                        logging.info("Retrying addition of dataset service account to {} (attempt #{})...".format(anvil_ingest_sa_group, str(attempt_counter)))
                        attempt_counter += 1
                        sleep(10)
                        continue
                    else:
                        add_text = " (Note that it may have already been added when the dataset SA was first created)" if not new_dataset else ""
                        error_message = "Error adding dataset ingest SA to {} group{}: {}".format(anvil_ingest_sa_group, add_text, res.text)
                        logging.error(error_message)
                        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Service Account Setup", "Add SA to Anvil Ingest Group", "Error", error_message])   
                        break
            except Exception as e:
                if attempt_counter < 2:
                        logging.info("Retrying addition of dataset service account to {} (attempt #{})...".format(anvil_ingest_sa_group, str(attempt_counter)))
                        attempt_counter += 1
                        sleep(10)
                        continue
                else:
                    add_text = " (Note that it may have already been added when the dataset SA was first created)" if not new_dataset else ""
                    error_message = "Error adding dataset ingest SA to {} group{}: {}".format(anvil_ingest_sa_group, add_text, str(e))
                    logging.error(error_message)
                    params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Service Account Setup", "Add SA to Anvil Ingest Group", "Error", error_message])   
                    break 
        # Add SA user to workspace as Reader
        acl_updates=[{"email": service_account,
                     "accessLevel":"READER",
                     "canShare":False}]
        attempt_counter = 0
        while True:
            try:
                res = update_workspace_acl(params["ws_project"], params["ws_name"], acl_updates=acl_updates)
                if res.status_code == 200:
                    logging.info("Dataset service account {} added to workspace as READER (without SHARE) permissions successfully.".format(service_account))
                    params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Service Account Setup", "Add SA to Workspace", "Success", service_account])
                    break
                else:
                    if attempt_counter < 2:
                        logging.info("Retrying addition of dataset service account to workspace (attempt #{})...".format(str(attempt_counter)))
                        attempt_counter += 1
                        sleep(10)
                        continue
                    else:
                        add_text = " (Note that it may have already been added when the dataset SA was first created)" if not new_dataset else ""
                        error_message = "Error adding dataset ingest SA to workspace{}: {}".format(add_text, res.text)
                        logging.error(error_message)
                        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Service Account Setup", "Add SA to Workspace", "Error", error_message])
                        break       
            except Exception as e:
                if attempt_counter < 2:
                    logging.info("Retrying addition of dataset service account to workspace (attempt #{})...".format(str(attempt_counter)))
                    attempt_counter += 1
                    sleep(10)
                    continue
                else:
                    add_text = " (Note that it may have already been added when the dataset SA was first created)" if not new_dataset else ""
                    error_message = "Error adding dataset ingest SA to workspace{}: {}".format(add_text, str(e))
                    logging.error(error_message)
                    params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Service Account Setup", "Add SA to Workspace", "Error", error_message])
                    break

# Function to create TDR dataset (with retry logic)
def create_tdr_dataset(dataset_id, params) -> str:
    logging.info("Creating new dataset: {}".format(params["dataset_name"]))
    # Setup/refresh TDR clients
    api_client = refresh_tdr_api_client()
    datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
    
    # Pull down TDR schema from cloud storage
    try:
        storage_client = storage.Client()
        bucket = storage_client.get_bucket(params["ws_bucket_name"])
        schema_blob = bucket.blob(params["el_schema_file"])
        schema = json.loads(schema_blob.download_as_string(client=None))
    except Exception as e:
        logging.error("Error retrieving TDR schema object: {}".format(str(e)))
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Creation or Retrieval", "Create New Dataset", "Error", "Error retrieving TDR schema object: {}".format(str(e))])
        return
    
    # Build and execute dataset creation request
    properties_dict = {
        "consent_name": params["consent_name"],
        "auth_domains": params["auth_domains"],
        "source_workspaces": params["source_workspaces"],
    }
    dataset_request = {
        "name": params["dataset_name"],
        "description": "TDR Dataset for {}".format(params["staging_area_name"]),
        "defaultProfileId": params["profile_id"],
        "region": "us-central1",
        "cloudPlatform": "gcp",
        "phsId": params["phs_id"],
        "experimentalSelfHosted": True,
        "dedicatedIngestServiceAccount": True,
        "experimentalPredictableFileIds": True,
        "properties": properties_dict,
        "schema": schema
    }
    attempt_counter = 0
    while True:
        try:
            create_dataset_result, job_id = wait_for_tdr_job(datasets_api.create_dataset(dataset=dataset_request))
            logging.info("Dataset Creation succeeded: {}".format(create_dataset_result))
            dataset_id = create_dataset_result["id"]
            params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Creation or Retrieval", "Create New Dataset", "Success", "Job_ID: {} - Truncated Response: {}".format(job_id, str(create_dataset_result)[0:1000])])
            try:
                resp = datasets_api.add_dataset_policy_member(id=dataset_id, policy_name="steward", policy_member={"email": "anvil_tdr_ingest@firecloud.org"})
            except:
                logging.warning("Error on adding additional policy members to dataset: {}".format(resp))
            return dataset_id
        except Exception as e:
            logging.error("Error on Dataset Creation: {}".format(str(e)))
            attempt_counter += 1
            if attempt_counter < 2:
                logging.info("Retrying Dataset Creation (attempt #{})...".format(str(attempt_counter)))
                sleep(10)
                continue
            else:
                logging.error("Maximum number of retries exceeded. Recording error to pipeline results.")
                params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Creation or Retrieval", "Create New Dataset", "Error", str(e)])
                return dataset_id

# Function to patch TDR dataset (with retry logic)
def patch_tdr_dataset(dataset_id, params) -> str:
    logging.info("Patching properties for existing dataset: {}".format(params["dataset_name"]))
    # Setup/refresh TDR clients
    api_client = refresh_tdr_api_client()
    datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
    
    # Retrieve existing data set properties
    try:
        dataset_info = datasets_api.retrieve_dataset(id=dataset_id, include=["PROPERTIES"]).to_dict()
        auth_domains = dataset_info["properties"]["auth_domains"]
        src_workspaces = dataset_info["properties"]["source_workspaces"]
        for ad in params["auth_domains"]:
            if ad not in auth_domains:
                auth_domains.append(ad)
        for sw in params["source_workspaces"]:
            if sw not in src_workspaces:
                src_workspaces.append(sw)
        properties_dict = {
            "consent_name": params["consent_name"],
            "auth_domains": auth_domains,
            "source_workspaces": src_workspaces
        }
    except:
        properties_dict = {
            "consent_name": params["consent_name"],
            "auth_domains": params["auth_domains"],
            "source_workspaces": params["source_workspaces"]
        }   
        
    # Execute dataset patch request (for phs_id and properties)
    attempt_counter = 0
    while True:
        try:
            phs_id = params["phs_id"]
            resp = datasets_api.patch_dataset(id=dataset_id, dataset_patch_request_model={"phsId": phs_id, "properties": properties_dict})
            params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Creation or Retrieval", "Patch Existing Dataset", "Success", resp])
            break
        except Exception as e:
            logging.error("Error on Dataset Patch: {}".format(str(e)))
            attempt_counter += 1
            if attempt_counter < 2:
                logging.info("Retrying Dataset Patch (attempt #{})...".format(str(attempt_counter)))
                sleep(10)
                continue
            else:
                logging.error("Maximum number of retries exceeded. Recording error to pipeline results.")
                params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Creation or Retrieval", "Patch Existing Dataset", "Error", str(e)])
                return
            
    # Retrieve current and target TDR schemas
    logging.info("Examining TDR schema to determine if new tables, columns, and/or relationships need to be added.")
    try:
        dataset_info = datasets_api.retrieve_dataset(id=dataset_id, include=["SCHEMA"]).to_dict()
        src_schema = {}
        src_schema["tables"] = dataset_info["schema"]["tables"]
        src_schema["relationships"] = dataset_info["schema"]["relationships"]
    except: 
        logging.error("Error retrieving source TDR schema: {}".format(str(e)))
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Creation or Retrieval", "Reconcile Existing Dataset Schema", "Error", "Error retrieving source TDR schema: {}".format(str(e))])
        return 
    try:
        storage_client = storage.Client()
        bucket = storage_client.get_bucket(params["ws_bucket_name"])
        schema_blob = bucket.blob(params["el_schema_file"])
        tar_schema = json.loads(schema_blob.download_as_string(client=None))
    except Exception as e:
        logging.error("Error retrieving target TDR schema object: {}".format(str(e)))
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Creation or Retrieval", "Reconcile Existing Dataset Schema", "Error", "Error retrieving target TDR schema object: {}".format(str(e))])
        return 
    
    # Compare current and target TDR schemas to determine if schema extension is needed (Tables)
    additional_tables = []
    for tar_table_entry in tar_schema["tables"]:
        add_table = True
        for src_table_entry in src_schema["tables"]:
            if tar_table_entry["name"] == src_table_entry["name"]:
                add_table = False
        if add_table == True:
            additional_tables.append(tar_table_entry)
    add_table_list = [table["name"] for table in additional_tables]
    
    # Compare current and target TDR schemas to determine if schema extension is needed (Columns)
    additional_columns = []
    for tar_table_entry in tar_schema["tables"]:
        if tar_table_entry["name"] not in add_table_list:
            add_col_list = []
            for tar_column_entry in tar_table_entry["columns"]:
                add_column = True
                for src_table_entry in src_schema["tables"]: 
                    for src_column_entry in src_table_entry["columns"]:
                        if tar_table_entry["name"] == src_table_entry["name"] and tar_column_entry["name"] == src_column_entry["name"]:
                            add_column = False
                if add_column == True:
                    add_col_list.append(tar_column_entry)
            if add_col_list:
                additional_columns.append({"tableName": tar_table_entry["name"], "columns": add_col_list})
    add_column_list = []
    for table_entry in additional_columns:
        for column_entry in table_entry["columns"]:
            add_column_list.append(table_entry["tableName"] + "." + column_entry["name"]) 
            
    # Compare current and target TDR schemas to determine if schema extension is needed (Relationships)
    additional_relationships = []
    for tar_rel_entry in tar_schema["relationships"]:
        add_rel = True
        for src_rel_entry in src_schema["relationships"]:
            if tar_rel_entry["name"] == src_rel_entry["name"]:
                add_rel = False
        if add_rel == True:
            additional_relationships.append(tar_rel_entry)
    add_rel_list = [rel["name"] for rel in additional_relationships]
    
    # Update TDR dataset schema to include additional tables and columns
    if not (add_table_list or add_column_list or add_rel_list):
        logging.info("No new tables, columns, or relationships to add to the TDR schema.")
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Creation or Retrieval", "Reconcile Existing Dataset Schema", "Success", "No new tables, columns, or relationships to add to the TDR schema."])
        return
    if add_table_list or add_column_list:
        logging.info("New tables and/or columns found. Building and submitting a TDR schema extension request.\n\tTables to add: {tabs}\n\tColumns to add: {cols}".format(tabs=add_table_list, cols=add_column_list))
        schema_update_request = {
            "description": "Adding tables and columns for additional source workspace",
            "changes": {
                "addTables": additional_tables,
                "addColumns": additional_columns
            }
        }
        attempt_counter = 0
        while True:
            try:
                schema_update_result, job_id = wait_for_tdr_job(datasets_api.update_schema(id=dataset_id, dataset_schema_update_model=schema_update_request))
                logging.info("TDR schema tables extension succeeded: {}".format(str(schema_update_result)[0:1000]))
                params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Creation or Retrieval", "Reconcile Existing Dataset Schema (Tables)", "Success", "Job_ID: {} - Truncated Response: {}".format(job_id, str(schema_update_result)[0:1000])])
                break
            except Exception as e:
                logging.error("Error on TDR schema extension: {}".format(str(e)))
                attempt_counter += 1
                if attempt_counter < 2:
                    logging.info("Retrying TDR schema extension (attempt #{})...".format(str(attempt_counter)))
                    sleep(15)
                    continue
                else:
                    logging.error("Maximum number of retries exceeded. Recording error to pipeline results.")
                    params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Creation or Retrieval", "Reconcile Existing Dataset Schema (Tables)", "Error", str(e)])
                    break
    if add_rel_list:
        logging.info("New relationships found. Building and submitting a TDR schema extension request.\n\tRelationships to add: {rels}".format(rels=add_rel_list))
        schema_update_request = {
            "description": "Adding relationships for additional source workspace",
            "changes": {
                "addRelationships": additional_relationships
            }
        }
        attempt_counter = 0
        while True:
            try:
                schema_update_result, job_id = wait_for_tdr_job(datasets_api.update_schema(id=dataset_id, dataset_schema_update_model=schema_update_request))
                logging.info("TDR schema relationships extension succeeded: {}".format(str(schema_update_result)[0:1000]))
                params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Creation or Retrieval", "Reconcile Existing Dataset Schema (Relationships)", "Success", "Job_ID: {} - Truncated Response: {}".format(job_id, str(schema_update_result)[0:1000])])
                break
            except Exception as e:
                logging.error("Error on TDR schema extension: {}".format(str(e)))
                attempt_counter += 1
                if attempt_counter < 2:
                    logging.info("Retrying TDR schema extension (attempt #{})...".format(str(attempt_counter)))
                    sleep(15)
                    continue
                else:
                    logging.error("Maximum number of retries exceeded. Recording error to pipeline results.")
                    params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Creation or Retrieval", "Reconcile Existing Dataset Schema (Relationships)", "Error", str(e)])
                    break
        
# Function to create or retrieve specified TDR dataset
def create_or_retrieve_dataset(params):
    logging.info("Attempting to create or retrieve the specified TDR dataset.")
    # Setup/refresh TDR clients
    api_client = refresh_tdr_api_client()
    datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
    
    # Attempt to identify dataset (if it already exists)
    dataset_id = ""
    new_dataset = False
    try:
        dataset_list = datasets_api.enumerate_datasets(filter=params["dataset_name"])
        if dataset_list.items:
            for dataset in dataset_list.items:
                if dataset.name == params["dataset_name"]:
                    dataset_id = str(dataset.id)
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Creation or Retrieval", "Enumerate Datasets", "Success", "{} datasets found. Matching dataset_id = {}".format(len(dataset_list.items), dataset_id)])
    except Exception as e:
        logging.error("Error on Dataset Enumeration: {}".format(str(e)))
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Creation or Retrieval", "Enumerate Datasets", "Error", str(e)])
        return
    if dataset_id == "":
        dataset_id = create_tdr_dataset(dataset_id, params)
        new_dataset = True
        return dataset_id, new_dataset
    else:
        logging.info("Dataset already exists! ID = {}".format(dataset_id))
        patch_tdr_dataset(dataset_id, params)
        return dataset_id, new_dataset

# Function to run file inventory build step, with retry logic
def run_build_file_inventory(params):
    inventory = []
    attempt_counter = 0
    while True:
        try:
            inventory, retry_count = bfi.build_inventory(params)
            logging.info("File Inventory build succeeded. {} distinct files found after deduplication. {} retries required.".format(len(inventory), retry_count))
            params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "File Inventory Creation", "Build File Inventory", "Success", "{} files found".format(len(inventory))])
            return inventory
        except Exception as e:
            logging.error("Error on File Inventory Creation: {}".format(str(e)))
            attempt_counter += 1
            if attempt_counter < 2:
                logging.info("Retrying File Inventory Creation (attempt #{})...".format(str(attempt_counter)))
                sleep(10)
                continue
            else:
                logging.error("Maximum number of retries exceeded. Recording error to pipeline results.")
                params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "File Inventory Creation", "Build File Inventory", "Error", str(e)])
                return inventory
    return inventory  

# Function to orchestrate the workspace connector pipeline
def run_ws_connector_pipeline(workspace, params):
    
    # Step 1: Set Variables for Pipeline
    workspace_name = workspace[0] 
    logging.info("Starting Workspace Connector Pipeline for {}.".format(workspace_name))
    ws_attributes = get_workspace_attributes(workspace[1], workspace[0])
    params["src_ws_name"] = workspace_name
    params["src_ws_project"] = workspace[1]
    params["phs_id"] = format_phs_id(ws_attributes["attributes"]["phs_id"]) if ws_attributes["attributes"].get("phs_id") else ""
    auth_list = ws_attributes["authorizationDomain"] if ws_attributes.get("authorizationDomain") else []
    params["auth_domains"] = [x["membersGroupName"] for x in auth_list]
    params["consent_name"] = ws_attributes["attributes"]["library:dataUseRestriction"] if ws_attributes["attributes"].get("library:dataUseRestriction") else ""
    params["staging_area_name"] = workspace[2]
    params["workspace_bucket"] = ws_attributes["bucketName"] if ws_attributes.get("bucketName") else "" 
    params["data_files_src_bucket"] = {}
    if params["workspace_bucket"]:
        params["data_files_src_bucket"][params["workspace_bucket"]] = {
            "include_dirs": [],
            "exclude_dirs": []
        }
    params["provenance_file"] = "ingest_pipeline/input/{}/provenance.json".format(params["staging_area_name"])
    params["input_dir"] = "ingest_pipeline/input/{}/table_data".format(params["staging_area_name"])
    params["file_inventory_dir"] = "ingest_pipeline/input/{}/data_files/file_inventory".format(params["staging_area_name"])
    current_datetime = datetime.datetime.now()
    current_datetime_string = current_datetime.strftime("%Y%m%d%H%M")
    params["pipeline_results"] = []
    logs_destination_dir = params["ws_bucket"] + "/ingest_pipeline/input/{}/logs".format(params["staging_area_name"])
    logs_stream_file_path = "pipeline_results_log_stream_" + current_datetime_string + ".txt"
    logs_table_file_path = "pipeline_results_log_" + current_datetime_string + ".tsv"
    while logging.root.handlers:
        logging.root.removeHandler(logging.root.handlers[-1])
    logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", datefmt="%m/%d/%Y %I:%M:%S %p", level=logging.INFO, handlers=[logging.FileHandler(logs_stream_file_path), logging.StreamHandler(sys.stdout)])

    # Step 2: Create or Update Provenance File
    logging.info("Creating or updating provenance.json file for Staging Area: {}".format(params["staging_area_name"]))
    try:
        storage_client = storage.Client()
        bucket = storage_client.get_bucket(params["ws_bucket_name"])
        provenance_blob = bucket.blob(params["provenance_file"])
        prov = json.loads(provenance_blob.download_as_string(client=None))
        prov["phs_id"] = params["phs_id"]
        prov["consent_name"] = params["consent_name"]
        if workspace_name not in prov["source_workspaces"]:
            prov["source_workspaces"].append(workspace_name)
        if params["data_files_src_bucket"]:
            for key in params["data_files_src_bucket"].keys():
                if key not in prov["data_files_src_buckets"].keys():
                       prov["data_files_src_buckets"][key] = params["data_files_src_bucket"][key]
        for entry in params["auth_domains"]:
            if entry not in prov["auth_domains"]:
                prov["auth_domains"].append(entry)
        logging.info("Existing provenance.json file found. Updating with new information.")
        fileref_list, prov["data_file_refs"], prov["data_files_src_buckets"], remote_bucket_list = find_and_add_fileref_fields(params["src_ws_project"], params["src_ws_name"], params["workspace_bucket"], params["data_file_refs"], prov["data_files_src_buckets"])
        if fileref_list:
            logging.info("Additional file reference fields found and marked for processing: " + ", ".join(fileref_list))
        if remote_bucket_list:
            warning_message = "References to files in remote buckets found. An attempt was made to add these to the data_files_src_buckets parameter for file inventory building. Ingest may not work as intended if the user and/or TDR service account does not have access to these remote buckets. Remote buckets include: " + ", ".join(remote_bucket_list)
            logging.warning(warning_message)
            params["pipeline_results"].append([params["src_ws_name"], params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Remote File References Check", "Warning", warning_message])
    except Exception as e:
        logging.info("Unable to retrieve provenance.json file. Creating new provenance.json file.")
        prov = {}
        prov["phs_id"] = params["phs_id"]
        prov["consent_name"] = params["consent_name"]
        prov["source_workspaces"] = [workspace_name]
        prov["auth_domains"] = params["auth_domains"]
        prov["data_files_src_buckets"] = params["data_files_src_bucket"]
        fileref_list, prov["data_file_refs"], prov["data_files_src_buckets"], remote_bucket_list = find_and_add_fileref_fields(params["src_ws_project"], params["src_ws_name"], params["workspace_bucket"], params["data_file_refs"], prov["data_files_src_buckets"])
        if fileref_list:
            logging.info("Additional file reference fields found and marked for processing: " + ", ".join(fileref_list))
        if remote_bucket_list:
            warning_message = "References to files in remote buckets found. An attempt was made to add these to the data_files_src_buckets parameter for file inventory building. Ingest may not work as intended if the user and/or TDR service account does not have access to these remote buckets. Remote buckets include: " + ", ".join(remote_bucket_list)
            logging.warning(warning_message)
            params["pipeline_results"].append([params["src_ws_name"], params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Remote File References Check", "Warning", warning_message])
    destination_file = "provenance.json"
    cloud_destination = params["ws_bucket"] + "/" + params["provenance_file"]
    with open(destination_file, "w") as outfile:
        json.dump(prov, outfile)
    !gsutil cp $destination_file $cloud_destination 2> stdout
    params["pipeline_results"].append([params["src_ws_name"], params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Create or Update Staging Area Provenance", "Success", json.dumps(prov)[0:1000]])
    
    # Step 3: Create Source Files
    if params["skip_source_files_creation"] == True:
        logging.info("Skipping source files creation.")
        params["pipeline_results"].append([params["src_ws_name"], params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Create Source Files", "Skipped", "User request"])
    else:
        logging.info("Running source files creation.")
        try:
            log_status, log_string = sfc.create_source_table_data_files(params)
            params["pipeline_results"].append([params["src_ws_name"], params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Create Source Files", log_status, log_string])
        except Exception as e:
            logging.error("Error creating source files: {}".format(str(e)))
            params["pipeline_results"].append([params["src_ws_name"], params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Create Source Files", "Error", str(e)]) 
    
    # Aggregate, save, and display pipeline results
    df_results = pd.DataFrame(params["pipeline_results"], columns = ["Workspace", "Staging Area", "Time", "Step", "Status", "Message"])
    logging.info("The Workspace Connector Pipeline has completed for {}.".format(workspace_name))
    copy_log_results(logs_destination_dir, logs_stream_file_path, logs_table_file_path, df_results)
    logging.info("Pipeline Results:")
    display(df_results)
    return 
    
# Function to orchestrate the various components of the ingest pipeline
def run_el_pipeline(pipeline, params): 
    
    # Step 1: Set Variables for Pipeline
    staging_area_name = pipeline[0] 
    logging.info("Starting Extract and Load (EL) Pipeline for {}.".format(staging_area_name))
    params["staging_area_name"] = staging_area_name
    params["dataset_name"] = pipeline[1]
    params["input_dir"] = "ingest_pipeline/input/{}/table_data".format(params["staging_area_name"])
    params["file_inventory_dir"] = "ingest_pipeline/input/{}/data_files/file_inventory".format(params["staging_area_name"])
    params["provenance_file"] = "ingest_pipeline/input/{}/provenance.json".format(params["staging_area_name"])
    params["el_output_dir"] = "ingest_pipeline/output/source/{}/table_data".format(params["staging_area_name"])
    params["el_schema_file"] = "ingest_pipeline/output/source/{}/schema/tdr_schema_object.json".format(params["staging_area_name"])
    current_datetime = datetime.datetime.now()
    current_datetime_string = current_datetime.strftime("%Y%m%d%H%M")
    params["snapshot_name"] = params["dataset_name"] + "_SRC_" + current_datetime_string
    params["pipeline_results"] = [] 
    logs_destination_dir = params["ws_bucket"] + "/ingest_pipeline/output/source/{}/logs".format(params["staging_area_name"])
    logs_stream_file_path = "pipeline_results_log_stream_" + current_datetime_string + ".txt"
    logs_table_file_path = "pipeline_results_log_" + current_datetime_string + ".tsv"
    while logging.root.handlers:
        logging.root.removeHandler(logging.root.handlers[-1])
    logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", datefmt="%m/%d/%Y %I:%M:%S %p", level=logging.INFO, handlers=[logging.FileHandler(logs_stream_file_path), logging.StreamHandler(sys.stdout)])
    
    # Step 2: Retrieve Staging Area Provenance Information
    try:
        storage_client = storage.Client()
        bucket = storage_client.get_bucket(params["ws_bucket_name"])
        provenance_blob = bucket.blob(params["provenance_file"])
        prov = json.loads(provenance_blob.download_as_string(client=None))
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Initialization", "Provenance File Retrieval", "Success", json.dumps(prov)[0:1000]])
    except Exception as e:
        logging.error("Error retrieving provenance.json file from staging area. Exiting pipeline.")
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Initialization", "Provenance File Retrieval", "Error", str(e)])
        df_results = pd.DataFrame(params["pipeline_results"], columns = ["Staging Area", "Time", "Step", "Task", "Status", "Message"])
        copy_log_results(logs_destination_dir, logs_stream_file_path, logs_table_file_path, df_results)
        logging.info("Pipeline Results:")
        display(df_results)
        return
    params["phs_id"] = prov["phs_id"] 
    params["consent_name"] = prov["consent_name"]
    params["source_workspaces"] = prov["source_workspaces"]
    params["data_files_src_buckets"] = prov["data_files_src_buckets"]
    if params["additional_file_inventory_sources"].get(params["staging_area_name"]):
        for key in params["additional_file_inventory_sources"][params["staging_area_name"]].keys():
            if key not in params["data_files_src_buckets"].keys():
                   params["data_files_src_buckets"][key] = params["additional_file_inventory_sources"][params["staging_area_name"]][key]
    params["auth_domains"] = prov["auth_domains"]
    params["data_file_refs"] = prov["data_file_refs"]
        
    # Step 3: Build File Inventory
    if params["skip_file_inventory_creation"] == True:
        logging.info("Skipping file inventory creation.")
        file_inventory = {}
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "File Inventory Creation", "Build File Inventory", "Skipped", "User request"])
    else:
        logging.info("Building file inventory.")
        file_inventory = run_build_file_inventory(params)
    params["file_inventory"] = file_inventory
    
    # Step 4: Process Table Data
    if params["skip_table_data_processing"] == True:
        logging.info("Skipping table data processing.")
        target_tables = {}
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Table Data Processing", "Ingest Pre-Processing", "Skipped", "User request"])
    else:
        logging.info("Processing table data for ingest.")
        target_tables, log_status, log_string = ptd.process_table_data(params)
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Table Data Processing", "Ingest Pre-Processing", log_status, log_string])
    
    # Step 5: Create or Retrieve Dataset
    int_df_results = pd.DataFrame(params["pipeline_results"], columns = ["Staging Area", "Time", "Step", "Task", "Status", "Message"])
    errors = int_df_results[int_df_results["Status"].str.contains("Error")]
    if len(errors) > 0:
        logging.info("Skipping TDR dataset creation or retrieval due to upstream errors. Exiting pipeline.")
        df_results = pd.DataFrame(params["pipeline_results"], columns = ["Staging Area", "Time", "Step", "Task", "Status", "Message"])
        copy_log_results(logs_destination_dir, logs_stream_file_path, logs_table_file_path, df_results)
        logging.info("Pipeline Results:")
        display(df_results)
        return 
    else:
        dataset_id, new_dataset = create_or_retrieve_dataset(params)
        if not dataset_id:
            logging.error("No TDR dataset created or retrieved. Exiting pipeline.")
            df_results = pd.DataFrame(params["pipeline_results"], columns = ["Staging Area", "Time", "Step", "Task", "Status", "Message"])
            copy_log_results(logs_destination_dir, logs_stream_file_path, logs_table_file_path, df_results)
            logging.info("Pipeline Results:")
            display(df_results)
            return 
        else:
            params["dataset_id"] = dataset_id
        if new_dataset == True:
            logging.info("Sleeping for a few minutes to let policy/permission changes propogate for new dataset...")
            sleep(60)
    
    # Step 6: Set up Dataset-specific Service Account
    set_up_dataset_ingest_sa(dataset_id, params, new_dataset)
    logging.info("Sleeping for a few minutes to let policy/permission changes propogate...")
    if new_dataset == True:
        sleep(300)
    else:
        sleep(60)
    
    # Step 7: Ingest Data to Dataset
    int_df_results = pd.DataFrame(params["pipeline_results"], columns = ["Staging Area", "Time", "Step", "Task", "Status", "Message"])
    errors = int_df_results[int_df_results["Status"].str.contains("Error")]
    if len(errors) > 0:
        logging.info("Skipping dataset ingests due to upstream errors.")
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Ingests", "Table: All - File: All", "Skipped", "Errors found upstream in ingest pipeline"])
    elif params["skip_ingests"] == True:
        logging.info("Skipping dataset ingests.")
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Ingests", "Table: All - File: All", "Skipped", "User request"])
    else:
        logging.info("Running dataset ingests.")
        if len(params["ingest_list_override"]) > 0:
            ingest_list = params["ingest_list_override"]
        else:
            ingest_list = {}
            for table in target_tables:
                ingest_list[table] = []
        run_el_ingests(dataset_id, params, ingest_list)   
    
    # Step 8: Create and Share Snapshot
    int_df_results = pd.DataFrame(params["pipeline_results"], columns = ["Staging Area", "Time", "Step", "Task", "Status", "Message"])
    errors = int_df_results[int_df_results["Status"].str.contains("Error")]
    if len(errors) > 0:
        logging.info("Skipping snapshot creation due to upstream errors.")
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Snapshot Creation", "Create and Share Snapshot", "Skipped", "Errors found upstream in ingest pipeline"])
    elif params["skip_snapshot_creation"] == True:
        logging.info("Skipping snapshot creation on user request.")
        params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Snapshot Creation", "Create and Share Snapshot", "Skipped", "User request"])
    else:
        logging.info("Running snapshot creation")
        try:
            create_and_share_snapshot(params, "el_pipeline")
        except Exception as e:
            logging.error("Error running snapshot creation: {}".format(str(e)))
            params["pipeline_results"].append([params["staging_area_name"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Snapshot Creation", "Create and Share Snapshot", "Error", str(e)])
    
    # Aggregate, save, and display pipeline results
    df_results = pd.DataFrame(params["pipeline_results"], columns = ["Staging Area", "Time", "Step", "Task", "Status", "Message"])
    logging.info("The ingest pipeline has completed for {}.".format(params["staging_area_name"]))
    copy_log_results(logs_destination_dir, logs_stream_file_path, logs_table_file_path, df_results)
    logging.info("Pipeline Results:")
    display(df_results)
    return


# "T" Pipeline Functions

In [ ]:
def derive_project_name(dataset_id, phs_id, dataset_name):
    project_name = ""
    # If PHS ID is provided, attempt to scrape from dbGaP
    if phs_id:
        try:
            phs_short = phs_id.replace("phs", "")
            dbgap_url = "https://dbgap.ncbi.nlm.nih.gov/ss/dbgapssws.cgi?request=Study&phs=" + phs_short
            http = urllib3.PoolManager()
            response = http.request('GET', dbgap_url)
            data = xmltodict.parse(response.data)
            if isinstance(data["dbgapss"]["Study"], list):
                project_name = data["dbgapss"]["Study"][0]["StudyInfo"]["StudyNameEntrez"]
            else:
                project_name = data["dbgapss"]["Study"]["StudyInfo"]["StudyNameEntrez"]
        except Exception as e:
            pass
    # If PHS ID is not provided or PHS ID doesn't produce a project name
    if not project_name:
        # Construct a query to pull project name from BigQuery, execute, and return results
        try:
            client = bigquery.Client()
            api_client = refresh_tdr_api_client()
            full_tdr_schema, bq_project, bq_schema, skip_bq_queries = odv.retrieve_tdr_schema(dataset_id, "dataset", api_client)
            proj_query = """SELECT COALESCE(MAX(CASE WHEN attribute = 'library:projectName' THEN NULLIF(value,'NA') END), MAX(CASE WHEN attribute = 'library:datasetName' THEN NULLIF(value,'NA') END)) AS project_name
                            FROM `{project}.{schema}.workspace_attributes`""".format(project = bq_project, schema = bq_schema)
            df = client.query(proj_query).result().to_dataframe()
            record_dict = df.to_dict(orient="records")
            project_name = record_dict[0]["project_name"]
        except Exception as e:
            pass
    # If no project name has been derived, set project name to dataset name
    if not project_name:
        project_name = dataset_name
    # Return project name
    return project_name

# Function to find and execute ingests
def run_t_ingests(params):
    dataset_id = params["dataset_id"]
    query_set = params["query_set"]
    override_list = params["ingest_list_override"]
    # Determine set of ingests to run
    ingest_list = query_set["transforms"].keys()
    filtered_ingest_list = []
    if override_list:
        for item in ingest_list:
            if item in override_list:
                filtered_ingest_list.append(item)
    else:
        filtered_ingest_list = ingest_list
    # Loop through and ingest files
    for item in filtered_ingest_list:
        target_table = item
        source_file_name = target_table + ".json" 
        source_rel_file_path = "{}/{}".format(params["t_output_dir"], source_file_name)
        source_full_file_path = "{}/{}/{}".format(params["ws_bucket"], params["t_output_dir"], source_file_name)
        if file_exists(source_rel_file_path, params):
            logging.info(f"Running ingest from {source_file_name} to table {target_table}.")
            ingest_request = {
                "table": target_table,
                "profile_id": params["profile_id"],
                "ignore_unknown_values": True,
                "resolve_existing_files": True,
                "updateStrategy": "replace",
                "format": "json",
                "bulkMode": True,
                "load_tag": "Ingest for {}".format(params["dataset_id"]),
                "path": source_full_file_path
            }
            attempt_counter = 0
            while True:
                try:
                    api_client = refresh_tdr_api_client()
                    datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
                    ingest_request_result, job_id = wait_for_tdr_job(datasets_api.ingest_dataset(id=dataset_id, ingest=ingest_request))
                    logging.info("Ingest from file {} succeeded: {}".format(source_file_name, str(ingest_request_result)[0:1000]))
                    params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Ingests", f"Table: {target_table} - File: {source_file_name}", "Success", "Job_ID: {} - Truncated Response: {}".format(job_id, str(ingest_request_result)[0:1000])])
                    break
                except Exception as e:
                    logging.error("Error on Dataset Ingest: {}".format(str(e)))
                    attempt_counter += 1
                    if attempt_counter < 2:
                        logging.info("Retrying Dataset Ingest (attempt #{})...".format(str(attempt_counter)))
                        sleep(10)
                        continue
                    else:
                        logging.error("Maximum number of retries exceeded. Recording error to pipeline results.")
                        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Ingests", f"Table: {target_table} - File: {source_file_name}", "Error", str(e)])
                        break
        else:
            logging.warning(f"Metadata file does not exist.  Skipping: {source_file_name}")
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Ingests", f"Table: {target_table} - File: {source_file_name}", "Warning", f"Metadata file does not exist.  Skipping: {source_file_name}"])
            continue              
                
# Function to extend schema of existing TDR dataset
def run_schema_extension(params):
    dataset_id = params["dataset_id"]
    target_schema = params["target_schema"]
    mapping_target = params["mapping_target"]
    # Retrieve current TDR schema and diff with target_schema
    logging.info("Retrieving current TDR schema to determine new tables and relationships to add.")
    api_client = refresh_tdr_api_client()
    datasets_api = data_repo_client.DatasetsApi(api_client=api_client)
    additional_tables = []
    additional_relationships = []
    try:
        response = datasets_api.retrieve_dataset(id=dataset_id, include=["SCHEMA", "ACCESS_INFORMATION"]).to_dict()
        current_table_list = [table["name"] for table in response["schema"]["tables"]]
        current_rel_list = [rel["name"] for rel in response["schema"]["relationships"]]
        for table_entry in target_schema["tables"]:
            if table_entry["name"] not in current_table_list:
                additional_tables.append(table_entry)
        for rel_entry in target_schema["relationships"]:
            if rel_entry["name"] not in current_rel_list:
                additional_relationships.append(rel_entry)
    except Exception as e:
        logging.warning("Error retrieving source schema from TDR. Will attempt to add all tables and relationships to schema. Error: {}".format(e))
        additional_tables = target_schema["tables"]
        additional_relationships = target_schema["relationships"]
    # Update TDR dataset schema to include additional tables and relationships
    add_table_list = [table["name"] for table in additional_tables]
    add_rel_list = [rel["name"] for rel in additional_relationships]
    if add_table_list or add_rel_list:
        logging.info("Submitting TDR schema extension request.\n\tTables to add: {tabs}\n\tRelationships to add: {rels}".format(tabs=add_table_list, rels=add_rel_list))
        schema_update_request = {
            "description": "Adding tables and relationships for mapping target: {}.".format(params["mapping_target"]),
            "changes": {
                "addTables": additional_tables,
                "addRelationships": additional_relationships
            }
        }
        attempt_counter = 0
        while True:
            try:
                schema_update_result, job_id = wait_for_tdr_job(datasets_api.update_schema(id=dataset_id, dataset_schema_update_model=schema_update_request))
                logging.info("TDR schema extension succeeded: {}".format(str(schema_update_result)[0:1000]))
                params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "TDR Schema Extension", "Extend TDR Schema", "Success", "Job_ID: {} - Truncated Response: {}".format(job_id, str(schema_update_result)[0:1000])])
                break
            except Exception as e:
                logging.error("Error on TDR schema extension: {}".format(str(e)))
                attempt_counter += 1
                if attempt_counter < 2:
                    logging.info("Retrying TDR schema extension (attempt #{})...".format(str(attempt_counter)))
                    sleep(10)
                    continue
                else:
                    logging.error("Maximum number of retries exceeded. Recording error to pipeline results.")
                    params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "TDR Schema Extension", "Extend TDR Schema", "Error", str(e)])
                    break
    else:
        logging.info("No new tables or relationships to add to the TDR schema.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "TDR Schema Extension", "Extend TDR Schema", "Success", "No new tables or relationships to add to the TDR schema."])
                
# Function to loop through and execute transform queries
def run_transform_queries(params):
    # Determine set of transforms to run
    query_set = params["query_set"]["transforms"]
    override_list = params["transform_list_override"]
    filtered_query_set = {}
    if override_list:
        for key, val in query_set.items():
            if key in override_list:
                filtered_query_set[key] = val
    else:
        filtered_query_set = query_set
    # Read in and execute queries
    client = bigquery.Client()
    for key, val in filtered_query_set.items():
        target_table = key
        target_file = target_table + ".json"
        destination_dir = params["t_output_dir"]
        logging.info("Running transform for target table: {}".format(target_table))
        query = val["query"]
        try:
            df = client.query(query).result().to_dataframe()
            records_json = df.to_json(orient='records') 
            records_list = json.loads(records_json)
            records_cnt = len(records_list)
            with open(target_file, 'w') as outfile:
                for idx, val in enumerate(records_list):
                    json.dump(val, outfile)
                    if idx < (records_cnt - 1):
                        outfile.write('\n')
            !gsutil cp $target_file $ws_bucket/$destination_dir/ 2> stdout
            !rm $target_file
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Transformed Files Creation", "File: {}".format(target_file), "Success", ""])
        except Exception as e:
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Transformed Files Creation", "File: {}".format(target_file), "Error", str(e)])
                
# Function to orchestrate the various components of the transformation ingest pipeline
def run_t_pipeline(params):
    
    # Step 1: Set Variables for Pipeline
    src_dataset = params["dataset_name"] + " ({})".format(params["dataset_id"])
    logging.info("Starting Transformation (T) Pipeline for {}.".format(src_dataset))
    params["src_dataset"] = src_dataset
    params["target_schema"] = {}
    params["query_set"] = {}
    storage_client = storage.Client()
    bucket = storage_client.get_bucket(params["ws_bucket_name"])
    blob = bucket.blob("ingest_pipeline/output/transformed/{}/{}/schema/mapping_schema_object.json".format(params["mapping_target"], params["dataset_id"]))
    params["target_schema"] = json.loads(blob.download_as_string(client=None))
    blob = bucket.blob("ingest_pipeline/output/transformed/{}/{}/queries/transform_query_set.json".format(params["mapping_target"], params["dataset_id"]))
    params["query_set"] = json.loads(blob.download_as_string(client=None))
    current_datetime = datetime.datetime.now()
    current_datetime_string = current_datetime.strftime("%Y%m%d%H%M")
    params["anvil_schema_version"] = "ANV" + str(params["target_schema"]["version"])
    params["snapshot_name"] = params["dataset_name"] + "_" + params["anvil_schema_version"] + "_" + current_datetime_string  
    params["t_output_dir"] = "ingest_pipeline/output/transformed/{}/{}/table_data".format(params["mapping_target"], params["dataset_id"])
    params["t_val_output_dir"] = "ingest_pipeline/output/transformed/{}/{}/validation".format(params["mapping_target"], params["dataset_id"])
    params["validation_schema_file"] = "ingest_pipeline/mapping/{}/mapping_schema_object.json".format(params["mapping_target"])
    params["pipeline_results"] = []
    logs_destination_dir = params["ws_bucket"] + "/ingest_pipeline/output/transformed/{}/{}/logs".format(params["mapping_target"], params["dataset_id"])
    logs_stream_file_path = "pipeline_results_log_stream_" + current_datetime_string + ".txt"
    logs_table_file_path = "pipeline_results_log_" + current_datetime_string + ".tsv"
    while logging.root.handlers:
        logging.root.removeHandler(logging.root.handlers[-1])
    logging.basicConfig(format="%(asctime)s - %(levelname)s: %(message)s", datefmt="%m/%d/%Y %I:%M:%S %p", level=logging.INFO, handlers=[logging.FileHandler(logs_stream_file_path), logging.StreamHandler(sys.stdout)])

    # Step 2: Confirm Transformation Artifacts
    logging.info("Attempting to confirm transform artifact retrieval (target schema and transform queries).")
    if params["target_schema"] and params["query_set"]:
        logging.info("Transform artifact retrieval confirmed.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Transform Artifact Retrieval", "Confirm Transform Artifact Retrieval", "Success", ""])
    else:
        if not params["target_schema"] and not params["query_set"]:
            error_message = "Target schema and transform queries both empty. Exiting pipeline."
            logging.info(error_message)
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Transform Artifact Retrieval", "Retrieve Target Schema and Transform Queries", "Error", error_message])
            df_results = pd.DataFrame(params["pipeline_results"], columns = ["Dataset", "Time", "Step", "Task", "Status", "Message"])
            copy_log_results(logs_destination_dir, logs_stream_file_path, logs_table_file_path, df_results)
            logging.info("Pipeline Results:")
            display(df_results)
            return
        elif not params["target_schema"]:
            error_message = "Target schema empty. Exiting pipeline."
            logging.info(error_message)
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Transform Artifact Retrieval", "Retrieve Target Schema and Transform Queries", "Error", error_message])
            df_results = pd.DataFrame(params["pipeline_results"], columns = ["Dataset", "Time", "Step", "Task", "Status", "Message"])
            copy_log_results(logs_destination_dir, logs_stream_file_path, logs_table_file_path, df_results)
            logging.info("Pipeline Results:")
            display(df_results)
            return
        else:
            error_message = "Transform queries empty. Exiting pipeline."
            logging.info(error_message)
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Transform Artifact Retrieval", "Retrieve Target Schema and Transform Queries", "Error", error_message])
            df_results = pd.DataFrame(params["pipeline_results"], columns = ["Dataset", "Time", "Step", "Task", "Status", "Message"])
            copy_log_results(logs_destination_dir, logs_stream_file_path, logs_table_file_path, df_results)
            logging.info("Pipeline Results:")
            display(df_results)
            return
                 
    # Step 3: Create Transformed Files
    if params["skip_transforms"] == True:
        logging.info("Skipping transformed files creation.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Transformed Files Creation", "File: All", "Skipped", "User request"])
    else:
        logging.info("Running transformed files creation.")
        run_transform_queries(params)
        
    # Step 4: Identify TDR Dataset and Extend Schema 
    if params["skip_schema_extension"] == True:
        logging.info("Skipping TDR schema extension.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "TDR Schema Extension", "Extend TDR Schema", "Skipped", "User request"])
    else:
        if len(params["dataset_id"]) == 0:
            logging.error("No TDR dataset specified. Exiting pipeline.")
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "TDR Schema Extension", "Extend TDR Schema", "Error", "No TDR dataset specified."])
            df_results = pd.DataFrame(params["pipeline_results"], columns = ["Dataset", "Time", "Step", "Task", "Status", "Message"])
            copy_log_results(logs_destination_dir, logs_stream_file_path, logs_table_file_path, df_results)
            logging.info("Pipeline Results:")
            display(df_results)
            return
        else:
            logging.info("Running TDR schema extension.")
            run_schema_extension(params)
            
    # Step 5: Ingest Transformed Files
    int_df_results = pd.DataFrame(params["pipeline_results"], columns = ["Dataset", "Time", "Step", "Task", "Status", "Message"])
    errors = int_df_results[int_df_results["Status"].str.contains("Error")]
    if len(errors) > 0:
        logging.info("Skipping dataset ingests due to upstream errors.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Ingests", "Table: All - File: All", "Skipped", "Errors found upstream in ingest pipeline"])
    elif params["skip_ingests"] == True:
        logging.info("Skipping dataset ingests.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dataset Ingests", "Table: All - File: All", "Skipped", "User request"])
    else:
        logging.info("Running dataset ingests")
        run_t_ingests(params)
    
    # Step 6: Infer File Relationships
    int_df_results = pd.DataFrame(params["pipeline_results"], columns = ["Dataset", "Time", "Step", "Task", "Status", "Message"])
    errors = int_df_results[int_df_results["Status"].str.contains("Error")]
    if len(errors) > 0:
        logging.info("Skipping file relationships inference due to upstream errors.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "File Relationship Inference", "Infer and Ingest File Relationships", "Skipped", "Errors found upstream in ingest pipeline"])
    elif params["skip_file_relation_inference"] == True:
        logging.info("Skipping file relationships inference.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "File Relationship Inference", "Infer and Ingest File Relationships", "Skipped", "User request"])
    else:
        logging.info("Running file relationships inference.")
        try:
            output, status = ifr.infer_file_relationships(params, params["dataset_id"])
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "File Relationship Inference", "Infer and Ingest File Relationships", status, output])
        except Exception as e:
            logging.error("Error running dangling foreign key resolution: {}".format(str(e)))
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "File Relationship Inference", "Infer and Ingest File Relationships", "Error", str(e)])   
    
    # Step 7: Remediate Dangling Foreign Keys
    int_df_results = pd.DataFrame(params["pipeline_results"], columns = ["Dataset", "Time", "Step", "Task", "Status", "Message"])
    errors = int_df_results[int_df_results["Status"].str.contains("Error")]
    if len(errors) > 0:
        logging.info("Skipping dangling foreign key resolution due to upstream errors.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dangling Foreign Key Resolution", "Resolve Dangling Foreign Keys", "Skipped", "Errors found upstream in ingest pipeline"])
    elif params["skip_dangling_fk_resolution"] == True:
        logging.info("Skipping dangling foreign key resolution.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dangling Foreign Key Resolution", "Resolve Dangling Foreign Keys", "Skipped", "User request"])
    else:
        logging.info("Running dangling foreign key resolution.")
        try:
            output, status = rdfk.resolve_dangling_fks(params, params["dataset_id"], params["target_schema"])
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dangling Foreign Key Resolution", "Resolve Dangling Foreign Keys", status, output])
        except Exception as e:
            logging.error("Error running dangling foreign key resolution: {}".format(str(e)))
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Dangling Foreign Key Resolution", "Resolve Dangling Foreign Keys", "Error", str(e)])   
    
    # Step 8: Identify supplementary data files
    int_df_results = pd.DataFrame(params["pipeline_results"], columns = ["Dataset", "Time", "Step", "Task", "Status", "Message"])
    errors = int_df_results[int_df_results["Status"].str.contains("Error")]
    if len(errors) > 0:
        logging.info("Skipping supplementary file identification due to upstream errors.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Supplementary File Identification", "Identify and Update Supplementary Files", "Skipped", "Errors found upstream in ingest pipeline"])
    elif params["skip_supplementary_file_identification"] == True:
        logging.info("Skipping supplementary file identification.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Supplementary File Identification", "Identify and Update Supplementary Files", "Skipped", "User request"])
    else:
        logging.info("Running supplementary file identification.")
        try:
            output, status = isf.identify_supplementary_files(params, params["dataset_id"])
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Supplementary File Identification", "Identify and Update Supplementary Files", status, output])
        except Exception as e:
            logging.error("Error running supplementary file identification: {}".format(str(e)))
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Supplementary File Identification", "Identify and Update Supplementary Files", "Error", str(e)])   
    
    # Step 9: Create and Share Snapshot
    int_df_results = pd.DataFrame(params["pipeline_results"], columns = ["Dataset", "Time", "Step", "Task", "Status", "Message"])
    errors = int_df_results[int_df_results["Status"].str.contains("Error")]
    if len(errors) > 0:
        logging.info("Skipping snapshot creation due to upstream errors.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Snapshot Creation", "Create and Share Snapshot", "Skipped", "Errors found upstream in ingest pipeline"])
    elif params["skip_snapshot_creation"] == True:
        logging.info("Skipping snapshot creation on user request.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Snapshot Creation", "Create and Share Snapshot", "Skipped", "User request"])
    else:
        logging.info("Running snapshot creation")
        try:
            create_and_share_snapshot(params, "t_pipeline")
        except Exception as e:
            logging.error("Error running snapshot creation: {}".format(str(e)))
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Snapshot Creation", "Create and Share Snapshot", "Error", str(e)])
    
    # Step 10: Data Profiling and Validation
    if len(errors) > 0:
        logging.info("Skipping output data validation due to upstream errors.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Output Data Validation", "Profile and Validate Data", "Skipped", "Errors found upstream in ingest pipeline"])
    elif params["skip_data_validation"] == True:
        logging.info("Skipping output data validation on user request.")
        params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Output Data Validation", "Profile and Validate Data", "Skipped", "User request"])
    else:
        logging.info("Running output data validation.")
        try:
            results_path = odv.profile_data(params["dataset_id"], "dataset", params["t_val_output_dir"], params["validation_schema_file"])
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Output Data Validation", "Profile and Validate Data", "Success", results_path])
        except Exception as e:
            logging.error("Error running output data validation: {}".format(str(e)))
            params["pipeline_results"].append([params["src_dataset"], datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S"), "Output Data Validation", "Profile and Validate Data", "Error", str(e)])   

    # Aggregate, save, and display pipeline results
    df_results = pd.DataFrame(params["pipeline_results"], columns = ["Dataset", "Time", "Step", "Task", "Status", "Message"])
    logging.info("The ingest pipeline has completed for {}.".format(params["src_dataset"]))
    copy_log_results(logs_destination_dir, logs_stream_file_path, logs_table_file_path, df_results)
    logging.info("Pipeline Results:")
    display(df_results)
    return
